<a href="https://colab.research.google.com/github/Pilla-Divya-Vidhyasree/leaf-disease-detection/blob/main/model.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
# IMPORTANT: SOME KAGGLE DATA SOURCES ARE PRIVATE
# RUN THIS CELL IN ORDER TO IMPORT YOUR KAGGLE DATA SOURCES.
import kagglehub
kagglehub.login()


In [ ]:
# IMPORTANT: RUN THIS CELL IN ORDER TO IMPORT YOUR KAGGLE DATA SOURCES,
# THEN FEEL FREE TO DELETE THIS CELL.
# NOTE: THIS NOTEBOOK ENVIRONMENT DIFFERS FROM KAGGLE'S PYTHON
# ENVIRONMENT SO THERE MAY BE MISSING LIBRARIES USED BY YOUR
# NOTEBOOK.

vipoooool_new_plant_diseases_dataset_path = kagglehub.dataset_download('vipoooool/new-plant-diseases-dataset')
pilladivya_vgg_tensorflow2_default_1_path = kagglehub.model_download('pilladivya/vgg/TensorFlow2/default/1')
pilladivya_inceptionv3_tensorflow2_default_1_path = kagglehub.model_download('pilladivya/inceptionv3/TensorFlow2/default/1')
pilladivya_inceptionv3_updated_tensorflow2_default_1_path = kagglehub.model_download('pilladivya/inceptionv3_updated/TensorFlow2/default/1')
pilladivya_ensemblelogic_keras_default_1_path = kagglehub.model_download('pilladivya/ensemblelogic/Keras/default/1')

print('Data source import complete.')


In [ ]:
from torchvision.datasets import ImageFolder
import torchvision.transforms as transforms
import random
from torch.utils.data import DataLoader
import torch
import torch.nn.functional as F
import torchvision.models as models
import torch.nn as nn
from torchvision.datasets import ImageFolder
from torchvision import transforms
import pandas as pd
import matplotlib.pyplot as plt

In [ ]:
import os

data_dir = '/kaggle/input/new-plant-diseases-dataset/New Plant Diseases Dataset(Augmented)/New Plant Diseases Dataset(Augmented)'

train_dir = data_dir + '/train'
test_dir = data_dir + '/valid'
data_names = os.listdir(train_dir)

In [ ]:
#printing diseasetypes
print(data_names)

In [ ]:
plants = []
NumberOfDiseases = 0
for plant in data_names:
    if plant.split('___')[0] not in plants:
        plants.append(plant.split('___')[0])
    if plant.split('___')[1] != 'healthy':
        NumberOfDiseases += 1

In [ ]:
# unique plants in the dataset
print(f"Unique Plants are: \n{plants}")

In [ ]:
# number of unique diseases
print("Number of diseases: {}".format(NumberOfDiseases))

In [ ]:
# Number of images for each disease
nums = {}
for disease in data_names:
    nums[disease] = len(os.listdir(train_dir + '/' + disease))

# converting the nums dictionary to pandas dataframe passing index as plant name and number of images as column

img_per_class = pd.DataFrame(nums.values(), index=nums.keys(), columns=["no. of images"])
img_per_class

In [ ]:
# plotting number of images available for each disease
index = [n for n in range(38)]
plt.figure(figsize=(20, 7))
plt.bar(index, [n for n in nums.values()], width=0.3)
plt.xlabel('Plants/Diseases', fontsize=10)
plt.ylabel('No of images available', fontsize=10)
plt.xticks(index, data_names, fontsize=12, rotation=90)
plt.title('Images per each class of plant disease')

In [ ]:
n_train = 0
for value in nums.values():
    n_train += value
print(f"There are {n_train} images for training")

In [ ]:
'''
# datasets for validation and training
train = ImageFolder(train_dir, transform=transforms.ToTensor())
valid = ImageFolder(valid_dir, transform=transforms.ToTensor()) '''

In [ ]:
'''img, label = train[0]
print(img.shape, label)'''

In [ ]:
'''
#Printing random image
random_img, random_label = train[random.randint(0,len(train)-1)]
plt.imshow(random_img.permute(1,2,0))
plt.axis('off')
plt.title(f'label: {train.classes[random_label]}')
plt.show()'''

In [ ]:
import tensorflow as tf
size = 224


train_generator=tf.keras.preprocessing.image.ImageDataGenerator(
    rescale=1/255.0,
    validation_split=0.1,
).flow_from_directory(train_dir,
                      batch_size=16,
                      target_size=(size,size),
                      subset="training",
                      color_mode='rgb',
                      class_mode='categorical',
                      shuffle=True)

In [ ]:
import numpy as np

classes=list(train_generator.class_indices.keys())
plt.figure(figsize=(20,20))
for X_batch, y_batch in train_generator:
    for i in range(0,16):
        plt.subplot(4,4,i+1)
        plt.imshow(X_batch[i])
        plt.title(classes[np.where(y_batch[i]==1)[0][0]])
    plt.show()
    break

In [ ]:
valid_generator=tf.keras.preprocessing.image.ImageDataGenerator(
    rescale=1/255.0,
    validation_split=0.1,
).flow_from_directory(train_dir,
                      batch_size=16,
                      target_size=(size,size),
                      subset='validation',
                      color_mode='rgb',
                      class_mode='categorical',
                      shuffle=False)

In [ ]:
test_generator=tf.keras.preprocessing.image.ImageDataGenerator(
    rescale=1/255.0,
).flow_from_directory(test_dir,
                      batch_size=16,
                      target_size=(size,size),
                      color_mode='rgb',
                      class_mode='categorical',
                      shuffle=False)

In [ ]:
classes=list(train_generator.class_indices.keys())
plt.figure(figsize=(20,20))
for X_batch, y_batch in test_generator:
    for i in range(0,16):
        plt.subplot(4,4,i+1)
        plt.imshow(X_batch[i])
        plt.title(classes[np.where(y_batch[i]==1)[0][0]])
    plt.show()
    break

In [ ]:
def Evaluate_model(model, train_generator, valid_generator, test_generator):
    model_evaluate_train = model.evaluate(train_generator)
    print("Loss of Train       : ",model_evaluate_train[0])
    print("Accuracy of Train   : ",model_evaluate_train[1])
    print("Precision of Train  : ",model_evaluate_train[2])
    print("Recall of Train     : ",model_evaluate_train[3])

    model_evaluate_valid = model.evaluate(valid_generator)
    print("Loss of Validation      : ",model_evaluate_valid[0])
    print("Accuracy of Validation  : ",model_evaluate_valid[1])
    print("Precision of Validation : ",model_evaluate_valid[2])
    print("Recall of Validation    : ",model_evaluate_valid[3])

    model_evaluate_test = model.evaluate(test_generator)
    print("Loss of Test      : ",model_evaluate_test[0])
    print("Accuracy of Test  : ",model_evaluate_test[1])
    print("Precision of Test : ",model_evaluate_test[2])
    print("Recall of Test    : ",model_evaluate_test[3])

    return np.round(model_evaluate_train[0], 2), np.round(model_evaluate_valid[0], 2), np.round(model_evaluate_test[0], 2), \
           np.round(model_evaluate_train[1], 2), np.round(model_evaluate_valid[1], 2), np.round(model_evaluate_test[1], 2), \
           np.round(model_evaluate_train[2], 2), np.round(model_evaluate_valid[2], 2), np.round(model_evaluate_test[2], 2), \
           np.round(model_evaluate_train[3], 2), np.round(model_evaluate_valid[3], 2), np.round(model_evaluate_test[3], 2)

In [ ]:
def draw_plots(model):
    acc = history.history['accuracy']
    val_acc = history.history['val_accuracy']

    precision = history.history['precision']
    val_precision = history.history['val_precision']

    recall =history.history['recall']
    val_recall =history.history['val_recall']

    loss = history.history['loss']
    val_loss = history.history['val_loss']

    epochs = range(1, len(loss) + 1)

    fig, ax = plt.subplots(2, 2, figsize=(17, 12))
    step = 2
        # Accuracy
    ax[0][0].plot(epochs, acc, color='green', label='Training Accuracy')
    ax[0][0].plot(epochs, val_acc, color='blue', label='Validation Accuracy')
    ax[0][0].set_title('Training and Validation Accuracy')
    ax[0][0].set_ylabel('Accuracy')
    ax[0][0].set_xlabel('Epoch')
    ax[0][0].legend()
    ax[0][0].set_xticks(np.arange(0, len(epochs) + step, step))
    ax[0][0].set_ylim(0, 1.02)

    # Precision
    ax[0][1].plot(epochs, precision, color='green', label='Training precision')
    ax[0][1].plot(epochs, val_precision, color='red', label='Validation precision')
    ax[0][1].set_title('Training and Validation precision')
    ax[0][1].set_xlabel('Epoch')
    ax[0][1].set_ylabel('precision')
    ax[0][1].legend()
    ax[0][1].set_xticks(np.arange(0, len(epochs) + step, step))
    ax[0][1].set_ylim(0,1.02)

    # Recall
    ax[1][0].plot(epochs, recall, color='green', label='Training Recall')
    ax[1][0].plot(epochs, val_recall, color='red', label='Validation Recall')
    ax[1][0].set_title('Training and Validation Recall')
    ax[1][0].set_xlabel('Epoch')
    ax[1][0].set_ylabel('Recall')
    ax[1][0].legend()
    ax[1][0].set_xticks(np.arange(0, len(epochs) + step, step))
    ax[1][0].set_ylim(0,1.02)

    # Loss
    ax[1][1].plot(epochs, loss, color='green', label='Training Loss')
    ax[1][1].plot(epochs, val_loss, color='red', label='Validation Loss')
    ax[1][1].set_title('Training and Validation Loss')
    ax[1][1].set_xlabel('Epoch')
    ax[1][1].set_ylabel('Loss')
    ax[1][1].legend()
    ax[1][1].set_xticks(np.arange(0, len(epochs) + step, step))

    plt.show()

In [ ]:
def Pridect_images (model, pred_dir, class_names):
    a=0
    for img in os.listdir(pred_dir):
        path = os.path.join(pred_dir, img)
        H,W,C = 224,224,3
        img = cv2.imread(path)
        img = cv2.resize(img, (H, W))
        img = cv2.cvtColor(img, cv2.COLOR_BGR2RGB)
        img = np.array(img)
        img = img.astype("float32")
        img = img / 255.0
        plt.imshow(img)
        img = np.expand_dims(img, axis=0)
        prediction = np.argmax(model.predict(img),axis=-1)[0]
        plt.title(f"predict : {class_names[prediction]}")
        plt.show()
        a+=1
        if a==5:
            break

In [ ]:
strategy = tf.distribute.MirroredStrategy()

In [ ]:
with strategy.scope():
    input_shape = (224, 224, 3)

    inputs = tf.keras.Input(shape=input_shape)

    base_model = tf.keras.applications.VGG19(include_top=False, weights='imagenet', input_tensor=inputs)

    base_model.trainable = False

    x = tf.keras.layers.Flatten()(base_model.output)
    x = tf.keras.layers.Dense(512, activation='relu')(x)
    x = tf.keras.layers.BatchNormalization()(x)
    x = tf.keras.layers.Dropout(0.3)(x)
    x = tf.keras.layers.Dense(256, activation='relu')(x)
    x = tf.keras.layers.BatchNormalization()(x)
    x = tf.keras.layers.Dropout(0.4)(x)
    x = tf.keras.layers.Dense(128, activation='relu')(x)
    x = tf.keras.layers.BatchNormalization()(x)
    x = tf.keras.layers.Dropout(0.5)(x)
    outputs = tf.keras.layers.Dense(38, activation='softmax')(x)

    model_VGG = tf.keras.models.Model(inputs = inputs, outputs = outputs)

    optimizer = tf.keras.optimizers.Adam(learning_rate=0.0001, beta_1=0.9, beta_2=0.999)

    model_VGG.compile(optimizer= optimizer, loss="categorical_crossentropy", metrics=["accuracy",'precision','recall'])

model_VGG.summary()

In [ ]:
from tensorflow.keras.callbacks import EarlyStopping, ModelCheckpoint,ReduceLROnPlateau

early_stopping = EarlyStopping(monitor='val_loss', patience=10, restore_best_weights=True)

model_checkpoint = ModelCheckpoint('best_model.keras', monitor='val_loss', save_best_only=True)

callbacks=[early_stopping,model_checkpoint]

In [ ]:
history = model_VGG.fit(train_generator,
                    epochs = 20,
                    batch_size=32,
                    validation_data = valid_generator,
                    callbacks = callbacks
                    )


In [ ]:
def load_latest_checkpoint(model, optimizer, scaler=None):
    """
    Load the latest checkpoint from the model save location, if available.
    """
    checkpoint_files = [f for f in os.listdir(model_save_loc) if f.startswith('checkpoint_epoch_') and f.endswith('.pth')]
    if not checkpoint_files:
        print("No checkpoints found. Starting training from scratch.")
        return 0, None

    # Find the latest checkpoint file by epoch number
    latest_file = max(checkpoint_files, key=lambda x: int(x.split('_')[-1].split('.')[0]))
    checkpoint_path = os.path.join(model_save_loc, latest_file)

    print(f"Loading checkpoint from: {checkpoint_path}")
    checkpoint = torch.load(checkpoint_path) #weights_only=True not needed
    model.load_state_dict(checkpoint['model_state_dict'])
    optimizer.load_state_dict(checkpoint['optimizer_state_dict'])
    if scaler and 'scaler' in checkpoint:
        scaler.load_state_dict(checkpoint['scaler'])
    start_epoch = checkpoint['epoch'] + 1  # Resume from the next epoch
    return start_epoch, checkpoint['loss']

In [ ]:
checkpoint_path = os.path.join(model_save_loc, f'checkpoint_epoch_{epoch}.pth')
        torch.save({
            'epoch': epoch,
            'model_state_dict': model.state_dict(),
            'optimizer_state_dict': optimizer.state_dict(),
            'scaler': scaler.state_dict(),  # Save the scaler state
            'loss': epoch_loss / len(train_loader.load_dataset()),
        }, checkpoint_path)
        print(f"Checkpoint saved at: {checkpoint_path}")

In [ ]:
Report_VGG = []
Report_VGG.append(Evaluate_model(model_VGG, train_generator, valid_generator, test_generator))
Report_VGG

In [ ]:
draw_plots(model_VGG)

In [ ]:
from sklearn.metrics import classification_report, confusion_matrix
import seaborn as sns

predictions = model_VGG.predict(test_generator)

y_pred = predictions.argmax(axis=-1)

class_names = list(test_generator.class_indices.keys())

print("Classification report : \n",classification_report(test_generator.labels, y_pred, target_names = class_names))

def draw_confusion_matrix(true, preds, normalize=None):
    conf_matx = confusion_matrix(true, preds, normalize = normalize)
    conf_matx = np.round(conf_matx, 2)

    sns.heatmap(conf_matx, annot=True, annot_kws={"size": 12},fmt="g", cbar=False, cmap="viridis")
    plt.show()

print("Confusion matrix : \n")
draw_confusion_matrix(test_generator.labels, y_pred)

print("Normalized confusion matrix : \n")
draw_confusion_matrix(test_generator.labels, y_pred, "true")

In [ ]:
model_VGG.save('/kaggle/working/VGG_Plant_Diseases.h5')
#!cp /kaggle/working/VGG_Plant_Diseases.h5 /kaggle/output/

In [ ]:
print(os.listdir('/kaggle/working'))




In [ ]:
model_VGG_load = tf.keras.models.load_model("/kaggle/input/vgg/tensorflow2/default/1/VGG_Plant_Diseases.h5")

In [ ]:
import cv2
pred_dir = '/kaggle/input/new-plant-diseases-dataset/test/test'
class_names = list(test_generator.class_indices.keys())
Pridect_images (model_VGG_load, pred_dir, class_names)

In [ ]:
with strategy.scope():
    input_shape = (224, 224, 3)

    inputs = tf.keras.Input(shape=input_shape)

    # Use ResNet50
    base_model = tf.keras.applications.ResNet50(include_top=False, weights='imagenet', input_tensor=inputs)

    base_model.trainable = False

    # Adjusting the following layers to fit the output of ResNet50
    x = tf.keras.layers.Flatten()(base_model.output)
    x = tf.keras.layers.Dense(512, activation='relu')(x)
    x = tf.keras.layers.BatchNormalization()(x)
    x = tf.keras.layers.Dropout(0.3)(x)
    x = tf.keras.layers.Dense(256, activation='relu')(x)
    x = tf.keras.layers.BatchNormalization()(x)
    x = tf.keras.layers.Dropout(0.4)(x)
    x = tf.keras.layers.Dense(128, activation='relu')(x)
    x = tf.keras.layers.BatchNormalization()(x)
    x = tf.keras.layers.Dropout(0.5)(x)
    outputs = tf.keras.layers.Dense(38, activation='softmax')(x)

    model_ResNet = tf.keras.models.Model(inputs=inputs, outputs=outputs)

    optimizer = tf.keras.optimizers.Adam(learning_rate=0.0001, beta_1=0.9, beta_2=0.999)

    model_ResNet.compile(optimizer=optimizer, loss="categorical_crossentropy", metrics=["accuracy", "precision", "recall"])

model_ResNet.summary()


In [ ]:
history = model_ResNet.fit(train_generator,
                    epochs = 20,
                    batch_size=32,
                    validation_data = valid_generator,
                    callbacks = callbacks
                    )

In [ ]:
Report_ResNet = []
Report_ResNet.append(Evaluate_model(model_ResNet, train_generator, valid_generator, test_generator))
Report_ResNet

In [ ]:
draw_plots(model_ResNet)

In [ ]:
from sklearn.metrics import classification_report, confusion_matrix

predictions = model_ResNet.predict(test_generator)

y_pred = predictions.argmax(axis=-1)

class_names = list(test_generator.class_indices.keys())

print("Classification report : \n",classification_report(test_generator.labels, y_pred, target_names = class_names))

def draw_confusion_matrix(true, preds, normalize=None):
    conf_matx = confusion_matrix(true, preds, normalize = normalize)
    conf_matx = np.round(conf_matx, 2)

    sns.heatmap(conf_matx, annot=True, annot_kws={"size": 12},fmt="g", cbar=False, cmap="viridis")
    plt.show()

print("Confusion matrix : \n")
draw_confusion_matrix(test_generator.labels, y_pred)

print("Normalized confusion matrix : \n")
draw_confusion_matrix(test_generator.labels, y_pred, "true")

In [ ]:
model_ResNet.save('/kaggle/working/ResNet_Plant_Diseases.h5')

In [ ]:
model_ResNet_load = tf.keras.models.load_model("/kaggle/working/ResNet_Plant_Diseases.h5")

In [ ]:
pred_dir = '/kaggle/input/new-plant-diseases-dataset/test/test'
class_names = list(test_generator.class_indices.keys())
Pridect_images (model_ResNet_load, pred_dir, class_names)

In [ ]:
with strategy.scope():
    input_shape = (224, 224, 3)
    inputs = tf.keras.Input(shape=input_shape)

    base_model = tf.keras.applications.DenseNet201(include_top=False, weights='imagenet', input_tensor=inputs)

    base_model.trainable = False

    x = tf.keras.layers.Flatten()(base_model.output)
    x = tf.keras.layers.Dense(512, activation='relu')(x)
    x = tf.keras.layers.BatchNormalization()(x)
    x = tf.keras.layers.Dropout(0.4)(x)
    x = tf.keras.layers.Dense(256, activation='relu')(x)
    x = tf.keras.layers.BatchNormalization()(x)
    x = tf.keras.layers.Dropout(0.3)(x)
    x = tf.keras.layers.Dense(128, activation='relu')(x)
    x = tf.keras.layers.BatchNormalization()(x)
    x = tf.keras.layers.Dropout(0.2)(x)
    outputs = tf.keras.layers.Dense(38, activation='softmax')(x)

    model_DenseNet = tf.keras.models.Model(inputs = inputs, outputs = outputs)

    optimizer = tf.keras.optimizers.Adam(learning_rate=0.001)

    model_DenseNet.compile(optimizer= optimizer, loss="categorical_crossentropy", metrics=["accuracy",'precision','recall'])

model_DenseNet.summary()

In [ ]:
from tensorflow.keras.callbacks import EarlyStopping, ModelCheckpoint,ReduceLROnPlateau

early_stopping = EarlyStopping(monitor='val_loss', patience=10, restore_best_weights=True)

model_ReduceLROnPlateau = ReduceLROnPlateau(monitor='val_loss', factor=0.1, patience=15, min_lr=0.000001)
model_checkpoint = ModelCheckpoint('best_model.keras', monitor='val_loss', save_best_only=True)
callbacks=[early_stopping,model_checkpoint,model_ReduceLROnPlateau]

In [ ]:
history = model_DenseNet.fit(train_generator,
                           epochs = 13,
                           batch_size=32,
                           validation_data = valid_generator,
                           callbacks = callbacks
                           )

In [ ]:
from sklearn.metrics import classification_report, confusion_matrix
import seaborn as sns
predictions = model_DenseNet.predict(test_generator)

y_pred = predictions.argmax(axis=-1)

class_names = list(test_generator.class_indices.keys())

print("Classification report : \n",classification_report(test_generator.labels, y_pred, target_names = class_names))

def draw_confusion_matrix(true, preds, normalize=None):
    conf_matx = confusion_matrix(true, preds, normalize = normalize)
    conf_matx = np.round(conf_matx, 2)

    sns.heatmap(conf_matx, annot=True, annot_kws={"size": 12},fmt="g", cbar=False, cmap="viridis")
    plt.show()

print("Confusion matrix : \n")
draw_confusion_matrix(test_generator.labels, y_pred)

print("Normalized confusion matrix : \n")
draw_confusion_matrix(test_generator.labels, y_pred, "true")

In [ ]:
model_DenseNet.save('/kaggle/working/DenseNet_Plant_Diseases.h5')

In [ ]:
Report_DenseNet = []
Report_DenseNet.append(Evaluate_model(model_DenseNet, train_generator, valid_generator, test_generator))
Report_DenseNet

In [ ]:
draw_plots(model_DenseNet_load)

In [ ]:
model_DenseNet_load = tf.keras.models.load_model("/kaggle/working/DenseNet_Plant_Diseases.h5")

In [ ]:
#Build InceptionV3 model
from tensorflow.keras.applications import InceptionV3
img_height, img_width = 224, 224
batch_size = 32

base_model = InceptionV3(weights='imagenet', include_top=False, input_shape=(img_height, img_width, 3))

for layer in base_model.layers:
    layer.trainable = False

In [ ]:
from tensorflow.keras.layers import GlobalAveragePooling2D
from tensorflow.keras.layers import Dense, Dropout, Flatten, Conv2D, MaxPooling2D
from tensorflow.keras.models import Model

x = base_model.output
x = GlobalAveragePooling2D()(x)  # Global average pooling layer
x = Dense(512, activation='relu')(x)  # Fully connected layer
predictions = Dense(len(classes), activation='softmax')(x)  # Final output layer
model = Model(inputs=base_model.input, outputs=predictions)

In [ ]:
model.summary()

In [ ]:
model.compile(
    loss="categorical_crossentropy",           # Use appropriate loss function for multi-class classification
    optimizer=tf.keras.optimizers.Adam(learning_rate=0.001),  # Choose optimizer and learning rate
    metrics=["accuracy", "precision", "recall"]  # Metrics to monitor
)

In [ ]:
from tensorflow.keras.callbacks import EarlyStopping, ReduceLROnPlateau, ModelCheckpoint

early_stopping=EarlyStopping(monitor='val_loss',patience=8,verbose=1, restore_best_weights=True)
checkpoints=ModelCheckpoint('resnet50.keras', monitor='val_loss', save_best_only=True,verbose=1)
reduceonplateau=ReduceLROnPlateau(monitor='val_loss', factor=0.1, patience=5, min_lr=0.001)
callback=[early_stopping,reduceonplateau,checkpoints]

In [ ]:
history =model.fit(train_generator,
                   epochs=15,
                   validation_data=valid_generator,
                   callbacks=callback,
                   batch_size=128)

In [ ]:
model.save('/kaggle/working/InceptionV3_Plant_Diseases.h5')

In [ ]:
Report_InceptionV3 = []
Report_InceptionV3.append(Evaluate_model(model, train_generator, valid_generator, test_generator))
Report_InceptionV3

In [ ]:
draw_plots(model)

In [ ]:
from sklearn.metrics import classification_report, confusion_matrix
import seaborn as sns
predictions = model.predict(test_generator)

y_pred = predictions.argmax(axis=-1)

class_names = list(test_generator.class_indices.keys())

print("Classification report : \n",classification_report(test_generator.labels, y_pred, target_names = class_names))

def draw_confusion_matrix(true, preds, normalize=None):
    conf_matx = confusion_matrix(true, preds, normalize = normalize)
    conf_matx = np.round(conf_matx, 2)

    sns.heatmap(conf_matx, annot=True, annot_kws={"size": 12},fmt="g", cbar=False, cmap="viridis")
    plt.show()

print("Confusion matrix : \n")
draw_confusion_matrix(test_generator.labels, y_pred)

print("Normalized confusion matrix : \n")
draw_confusion_matrix(test_generator.labels, y_pred, "true")

In [ ]:
with strategy.scope():
    input_shape = (224, 224, 3)
    inputs = tf.keras.Input(shape=input_shape)

    base_model = tf.keras.applications.InceptionV3(include_top=False, weights='imagenet', input_tensor=inputs)

    base_model.trainable = False

    x = tf.keras.layers.Flatten()(base_model.output)
    x = tf.keras.layers.Dense(512, activation='relu')(x)
    x = tf.keras.layers.BatchNormalization()(x)
    x = tf.keras.layers.Dropout(0.4)(x)
    x = tf.keras.layers.Dense(256, activation='relu')(x)
    x = tf.keras.layers.BatchNormalization()(x)
    x = tf.keras.layers.Dropout(0.3)(x)
    x = tf.keras.layers.Dense(128, activation='relu')(x)
    x = tf.keras.layers.BatchNormalization()(x)
    x = tf.keras.layers.Dropout(0.2)(x)
    outputs = tf.keras.layers.Dense(38, activation='softmax')(x)

    model_InceptionV3 = tf.keras.models.Model(inputs = inputs, outputs = outputs)

    optimizer = tf.keras.optimizers.Adam(learning_rate=0.001)

    model_InceptionV3.compile(optimizer= optimizer, loss="categorical_crossentropy", metrics=["accuracy",'precision','recall'])

model_InceptionV3.summary()

In [ ]:
from tensorflow.keras.callbacks import EarlyStopping, ModelCheckpoint,ReduceLROnPlateau

early_stopping = EarlyStopping(monitor='val_loss', patience=10, restore_best_weights=True)

model_ReduceLROnPlateau = ReduceLROnPlateau(monitor='val_loss', factor=0.1, patience=15, min_lr=0.000001)
model_checkpoint = ModelCheckpoint('best_model.keras', monitor='val_loss', save_best_only=True)
callbacks=[early_stopping,model_checkpoint,model_ReduceLROnPlateau]

In [ ]:
history = model_InceptionV3.fit(train_generator,
                           epochs = 15,
                           batch_size=32,
                           validation_data = valid_generator,
                           callbacks = callbacks
                           )

In [ ]:
model_InceptionV3.save('/kaggle/working/InceptionV3_Plant_Diseases_updated.h5')

In [ ]:
Report_of_InceptionV3 = []
Report_of_InceptionV3.append(Evaluate_model(model_InceptionV3, train_generator, valid_generator, test_generator))
Report_of_InceptionV3

In [ ]:
draw_plots(model_InceptionV3)

In [ ]:
from sklearn.metrics import classification_report, confusion_matrix
import seaborn as sns
predictions = model_InceptionV3.predict(test_generator)

y_pred = predictions.argmax(axis=-1)

class_names = list(test_generator.class_indices.keys())

print("Classification report : \n",classification_report(test_generator.labels, y_pred, target_names = class_names))

def draw_confusion_matrix(true, preds, normalize=None):
    conf_matx = confusion_matrix(true, preds, normalize = normalize)
    conf_matx = np.round(conf_matx, 2)

    sns.heatmap(conf_matx, annot=True, annot_kws={"size": 12},fmt="g", cbar=False, cmap="viridis")
    plt.show()

print("Confusion matrix : \n")
draw_confusion_matrix(test_generator.labels, y_pred)

print("Normalized confusion matrix : \n")
draw_confusion_matrix(test_generator.labels, y_pred, "true")

In [ ]:
'''import numpy as np
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import accuracy_score, precision_score, recall_score
from tensorflow.keras.models import load_model

# Step 1: Load the trained models
model_inception = load_model("/kaggle/input/inceptionv3_updated/tensorflow2/default/1/InceptionV3_Plant_Diseases_updated.h5")
model_densenet = load_model("/kaggle/input/inceptionv3/tensorflow2/default/1/DenseNet_Plant_Diseases.h5")
model_vgg = load_model("/kaggle/input/vgg/tensorflow2/default/1/VGG_Plant_Diseases.h5")

# Step 2: Extract predictions from each model as features
def get_predictions(model, generator):
    # Predict probabilities for all images in the generator
    return model.predict(generator)

# Train predictions
train_preds_inception = get_predictions(model_inception, train_generator)
train_preds_densenet = get_predictions(model_densenet, train_generator)
train_preds_vgg = get_predictions(model_vgg, train_generator)

# Validation predictions
val_preds_inception = get_predictions(model_inception, valid_generator)
val_preds_densenet = get_predictions(model_densenet, valid_generator)
val_preds_vgg = get_predictions(model_vgg, valid_generator)

# Test predictions
test_preds_inception = get_predictions(model_inception, test_generator)
test_preds_densenet = get_predictions(model_densenet, test_generator)
test_preds_vgg = get_predictions(model_vgg, test_generator)

# Step 3: Combine predictions as features for stacking
def combine_features(pred1, pred2, pred3):
    return np.hstack([pred1, pred2, pred3])

# Combine for training
train_features = combine_features(train_preds_inception, train_preds_densenet, train_preds_vgg)
val_features = combine_features(val_preds_inception, val_preds_densenet, val_preds_vgg)
test_features = combine_features(test_preds_inception, test_preds_densenet, test_preds_vgg)

# Step 4: Extract true labels from the generator
# Assuming the file names are used to create class indices
train_labels = train_generator.classes
val_labels = valid_generator.classes
test_labels = test_generator.classes

# Step 5: Train Logistic Regression as meta-classifier
meta_classifier = LogisticRegression(max_iter=500)  # Increase max_iter if needed
meta_classifier.fit(train_features, train_labels)

# Step 6: Evaluate on Validation and Test Sets
val_predictions = meta_classifier.predict(val_features)
test_predictions = meta_classifier.predict(test_features)

# Validation metrics
val_accuracy = accuracy_score(val_labels, val_predictions)
val_precision = precision_score(val_labels, val_predictions, average="weighted")
val_recall = recall_score(val_labels, val_predictions, average="weighted")

print(f"Validation Accuracy: {val_accuracy}")
print(f"Validation Precision: {val_precision}")
print(f"Validation Recall: {val_recall}")

# Test metrics
test_accuracy = accuracy_score(test_labels, test_predictions)
test_precision = precision_score(test_labels, test_predictions, average="weighted")
test_recall = recall_score(test_labels, test_predictions, average="weighted")

print(f"Test Accuracy: {test_accuracy}")
print(f"Test Precision: {test_precision}")
print(f"Test Recall: {test_recall}")
'''

In [ ]:
'''from sklearn.ensemble import RandomForestClassifier

# Update stacking model
meta_model = RandomForestClassifier(n_estimators=100, random_state=42)
meta_model.fit(train_features, train_labels)'''


In [ ]:
'''# Step 6: Evaluate on Validation and Test Sets
val_predictions = meta_model.predict(val_features)
test_predictions = meta_model.predict(test_features)

# Validation metrics
val_accuracy = accuracy_score(val_labels, val_predictions)
val_precision = precision_score(val_labels, val_predictions, average="weighted")
val_recall = recall_score(val_labels, val_predictions, average="weighted")

print(f"Validation Accuracy: {val_accuracy}")
print(f"Validation Precision: {val_precision}")
print(f"Validation Recall: {val_recall}")

# Test metrics
test_accuracy = accuracy_score(test_labels, test_predictions)
test_precision = precision_score(test_labels, test_predictions, average="weighted")
test_recall = recall_score(test_labels, test_predictions, average="weighted")

print(f"Test Accuracy: {test_accuracy}")
print(f"Test Precision: {test_precision}")
print(f"Test Recall: {test_recall}")'''

In [ ]:
# Get predictions from all three models
y_pred1 = model_inception.predict(test_generator)  # Shape: (num_samples, num_classes)
y_pred2 = model_vgg.predict(test_generator)  # Shape: (num_samples, num_classes)
y_pred3 = model_densenet.predict(test_generator)  # Shape: (num_samples, num_classes)

# Average the probabilities
final_probabilities = (y_pred1 + y_pred2 + y_pred3) / 3

# Get final predictions (class with the highest probability)
final_predictions = final_probabilities.argmax(axis=1)

# Map predictions to class labels
class_labels = list(test_generator.class_indices.keys())
final_classes = [class_labels[pred] for pred in final_predictions]

# Save predictions to evaluate accuracy or other metrics
from sklearn.metrics import accuracy_score, classification_report

# True labels
true_labels = test_generator.classes

# Accuracy and classification report
print("Accuracy:", accuracy_score(true_labels, final_predictions))
print("Classification Report:\n", classification_report(true_labels, final_predictions))


In [ ]:
with open('ensemble_logic.py', 'w') as f:
    model1 = load_model("/kaggle/input/inceptionv3_updated/tensorflow2/default/1/InceptionV3_Plant_Diseases_updated.h5")
    model2 = load_model("/kaggle/input/inceptionv3/tensorflow2/default/1/DenseNet_Plant_Diseases.h5")
    model3 = load_model("/kaggle/input/vgg/tensorflow2/default/1/VGG_Plant_Diseases.h5")
    def ensemble_predict(image):
        class_mapping ={
        0: 'Apple___Apple_scab',
        1: 'Apple___Black_rot',
        2: 'Apple___Cedar_apple_rust',
        3: 'Apple___healthy', 4: 'Blueberry___healthy',
        5: 'Cherry_(including_sour)___Powdery_mildew', 6: 'Cherry_(including_sour)___healthy',
        7: 'Corn_(maize)___Cercospora_leaf_spot Gray_leaf_spot', 8: 'Corn_(maize)___Common_rust_',
        9: 'Corn_(maize)___Northern_Leaf_Blight', 10: 'Corn_(maize)___healthy',
        11: 'Grape___Black_rot', 12: 'Grape___Esca_(Black_Measles)',
        13: 'Grape___Leaf_blight_(Isariopsis_Leaf_Spot)', 14: 'Grape___healthy',
        15: 'Orange___Haunglongbing_(Citrus_greening)', 16: 'Peach___Bacterial_spot',
        17: 'Peach___healthy', 18: 'Pepper,_bell___Bacterial_spot', 19: 'Pepper,_bell___healthy',
        20: 'Potato___Early_blight', 21: 'Potato___Late_blight', 22: 'Potato___healthy',
        23: 'Raspberry___healthy', 24: 'Soybean___healthy', 25: 'Squash___Powdery_mildew',
        26: 'Strawberry___Leaf_scorch', 27: 'Strawberry___healthy', 28: 'Tomato___Bacterial_spot',
        29: 'Tomato___Early_blight', 30: 'Tomato___Late_blight', 31: 'Tomato___Leaf_Mold',
        32: 'Tomato___Septoria_leaf_spot', 33: 'Tomato___Spider_mites Two-spotted_spider_mite',
        34: 'Tomato___Target_Spot', 35: 'Tomato___Tomato_Yellow_Leaf_Curl_Virus',
        36: 'Tomato___Tomato_mosaic_virus',
        37: 'Tomato___healthy'
        }
        # Get predictions from each model
        pred1 = model1.predict(image)
        pred2 = model2.predict(image)
        pred3 = model3.predict(image)

        # Average the predictions
        final_probabilities = (pred1 + pred2 + pred3) / 3

        # Get the predicted class
        final_prediction = np.argmax(final_probabilities, axis=1)
        # Extract the scalar value from the ndarray (which will have a single element)
        predicted_class_index = final_prediction[0]  # Get the scalar value (first element)
        predicted_class_name = class_mapping.get(predicted_class_index, "Unknown Class")
        return predicted_class_name

In [ ]:
class_mapping = {v: k for k, v in train_generator.class_indices.items()}

print(class_mapping)

In [ ]:
import os
import random
from tensorflow.keras.utils import load_img, img_to_array
import numpy as np

# Define the test directory path
test_dir = '/kaggle/input/new-plant-diseases-dataset/test/'

# Get a random class folder
class_folders = [os.path.join(test_dir, folder) for folder in os.listdir(test_dir)]
random_class_folder = random.choice(class_folders)

# Get a random image from the selected class folder
sample_image_path = os.path.join(random_class_folder, random.choice(os.listdir(random_class_folder)))
print(f"Sample Image Path: {sample_image_path}")

# Load and preprocess the image
def preprocess_image(image_path, target_size=(224, 224)):
    image = load_img(image_path, target_size=target_size)  # Resize image
    image_array = img_to_array(image)  # Convert to array
    image_array = np.expand_dims(image_array, axis=0)  # Add batch dimension
    image_array = image_array / 255.0  # Normalize to [0, 1]
    return image_array

preprocessed_image = preprocess_image(sample_image_path)


In [ ]:
print(ensemble_predict(preprocessed_image))

In [ ]:
'''

model_evaluate_train = model_inception.evaluate(train_generator)
print("Loss of Train       : ",model_evaluate_train[0])
print("Accuracy of Train   : ",model_evaluate_train[1])
print("Precision of Train  : ",model_evaluate_train[2])
print("Recall of Train     : ",model_evaluate_train[3])'''

In [ ]:
# Define batch size
batch_size = 32

# Create DataLoaders
train_loader = DataLoader(train, batch_size=batch_size, num_workers=2, shuffle=True)
valid_loader = DataLoader(valid, batch_size=batch_size,num_workers=2, shuffle=False)

In [ ]:
from tensorflow.keras.applications import ResNet50, InceptionV3, VGG16
from tensorflow.keras.layers import Input, GlobalAveragePooling2D, Dense, Concatenate, Dropout
from tensorflow.keras.models import Model
from tensorflow.keras.optimizers import Adam

# Define the number of classes
num_classes = 38
input_shape = (224, 224, 3)

# ResNet50 Base Model
resnet_base = ResNet50(weights='imagenet', include_top=False, input_shape=input_shape)
for layer in resnet_base.layers:
    layer.trainable = False  # Freeze layers
resnet_out = GlobalAveragePooling2D()(resnet_base.output)

# InceptionV3 Base Model
inception_base = InceptionV3(weights='imagenet', include_top=False, input_shape=input_shape)
for layer in inception_base.layers:
    layer.trainable = False  # Freeze layers
inception_out = GlobalAveragePooling2D()(inception_base.output)

# VGG16 Base Model
vgg_base = VGG16(weights='imagenet', include_top=False, input_shape=input_shape)
for layer in vgg_base.layers:
    layer.trainable = False  # Freeze layers
vgg_out = GlobalAveragePooling2D()(vgg_base.output)

In [ ]:
# Combine the outputs from the three models
combined = Concatenate()([resnet_out, inception_out, vgg_out])


In [ ]:
# Fully connected layers after concatenation
x = Dense(512, activation='relu')(combined)  # First fully connected layer
x = Dropout(0.5)(x)  # Add dropout to prevent overfitting
x = Dense(256, activation='relu')(x)  # Second fully connected layer
x = Dropout(0.3)(x)  # Add another dropout layer

# Final classification layer with Softmax
output = Dense(num_classes, activation='softmax')(x)

# Define the ensemble model
ensemble_model = Model(inputs=[resnet_base.input, inception_base.input, vgg_base.input], outputs=output)


In [ ]:
# Compile the model
ensemble_model.compile(optimizer=Adam(learning_rate=0.001),
                       loss='categorical_crossentropy',
                       metrics=['accuracy'])

# Display model summary
ensemble_model.summary()

In [ ]:
from torch.utils.data import random_split

# Define the sizes for validation and test datasets
valid_size = int(0.5 * len(valid))  # 50% for validation
test_size = len(valid) - valid_size  # Remaining 50% for testing

# Split the dataset
valid_images, test_images = random_split(valid, [valid_size, test_size])


In [ ]:
from torch.utils.data import DataLoader

# Hyperparameters
batch_size = 32

# DataLoaders for train, validation, and test sets
train_loader = DataLoader(train, batch_size=batch_size, shuffle=True)
valid_loader = DataLoader(valid_images, batch_size=batch_size, shuffle=True)
test_loader = DataLoader(test_images, batch_size=batch_size, shuffle=False)


In [ ]:
import numpy as np

#Function to convert PyTorch DataLoader to NumPy arrays
def dataloader_to_numpy(dataloader):
    images = []
    labels = []
    for batch in dataloader:
        x, y = batch
        images.append(x.numpy())  # Convert images to numpy
        labels.append(y.numpy())  # Convert labels to numpy
    images = np.concatenate(images)  # Combine all batches
    labels = np.concatenate(labels)
    return images, labels

# Convert DataLoaders to Numpy
train_images, train_labels = dataloader_to_numpy(train_loader)
valid_images, valid_labels = dataloader_to_numpy(valid_loader)
test_images, test_labels = dataloader_to_numpy(test_loader)

In [ ]:
def load_images_from_dir(directory):
    image_paths = []
    for root, dirs, files in os.walk(directory):
        for file in files:
            if file.lower().endswith(('.png', '.jpg', '.jpeg')):  # Add other extensions if needed
                image_paths.append(os.path.join(root, file))
    return image_paths

# Extract images
train_images = load_images_from_dir(train_dir)
val_images = load_images_from_dir(valid_dir)

# If test images are part of the valid_dir, adjust path accordingly
test_dir = os.path.join(valid_dir, 'test')  # Update if necessary
test_images = load_images_from_dir(test_dir)

# Summary
print(f"Number of training images: {len(train_images)}")
print(f"Number of validation images: {len(val_images)}")
print(f"Number of testing images: {len(test_images)}")


train_images = train_images / 255.0
valid_images = valid_images / 255.0
test_images = test_images / 255.0


In [ ]:
# Fit the ensemble model
history = ensemble_model.fit(
    [train_loader, train_loader, train_loader],  # Inputs for ResNet, Inception, VGG
    train_labels,  # True labels
    validation_data=([val_loader, val_loader, val_loader], val_labels),  # Validation data
    batch_size=32,
    epochs=20
)


In [ ]:
test_loss, test_accuracy = ensemble_model.evaluate(
    [test_images, test_images, test_images], test_labels, batch_size=32
)

print(f"Test Accuracy: {test_accuracy * 100:.2f}%")
